Three observations about the data:

1.) The player base is primarily male. This would probably be considered expected for data concerning video games. 
2.) Retribution Axe is not the highest priced item but does produce the highes profit for the company. 
3.) Overall, higher priced items produce the most profit for the company. The company should look into why this is and how to drive customers towards these higher priced items. 

In [1]:
import pandas as pd
import numpy as np
import json

In [3]:
json_path=("purchase_data.json")
purchase_data_df = pd.read_json(json_path, orient="records")
purchase_data_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [6]:
player_count = len(purchase_data_df['SN'].unique())
player_count

573

In [5]:
player_count_df = pd.DataFrame({"Number of Players":[player_count]})
player_count_df

,Number of Players
0,573


In [9]:
item_count=len(purchase_data_df['Item Name'].unique())
item_count

avg_purchase_price = round(purchase_data_df['Price'].mean(), 2)
avg_purchase_price

total_purchases = len(purchase_data_df['Price'])
total_purchases

total_revenue = round(purchase_data_df['Price'].sum(), 2)
total_revenue

2286.33

In [10]:
purchase_analysis_df = pd.DataFrame({"Number of Items":[item_count], "Average Purchase Price":[avg_purchase_price], "Total Purchases":[total_purchases], "Total Revenue":[total_revenue]})
purchase_analysis_df

,Average Purchase Price,Number of Items,Total Purchases,Total Revenue
0,2.93,179,780,2286.33


In [41]:
unique_players_df = purchase_data_df.drop_duplicates('SN')
len(unique_players_df)

573

In [44]:
gender_count_df = pd.DataFrame(unique_players_df['Gender'].value_counts())
gender_count_df.rename(columns={'Gender':'Number of Players'}, inplace=True)
gender_count_df

,Number of Players
Male,465
Female,100
Other / Non-Disclosed,8


In [72]:
gender_percent_df = round(gender_count_df/player_count*100)
combined_gender_data=pd.merge(gender_count_df, gender_percent_df, right_index=True, left_index=True)
combined_gender_data.rename(columns = {'Number of Players_x': 'Number of Players', 'Number of Players_y': 'Percentage of Total Players'}, inplace=True)
combined_gender_data

,Number of Players,Percentage of Total Players
Male,465,81.0
Female,100,17.0
Other / Non-Disclosed,8,1.0


In [8]:
gender_purchase_data_df = purchase_data_df.set_index(['Gender'])
gender_norm=purchase_data_df.groupby(['Gender'])['Price']
gender_price=purchase_data_df.groupby(['Gender'])['Price'].sum()
gender_purchase=purchase_data_df.groupby(['Gender'])['Price'].count()
gender_norm_mean=purchase_data_df.groupby(['Gender'])['Price'].mean()
avg_gen= round(gender_price/gender_purchase,2)
gender_norm_max=purchase_data_df.groupby(['Gender'])['Price'].max()
gender_norm_min=purchase_data_df.groupby(['Gender'])['Price'].min()
gender_norm_max.index
gender_norm_df=(gender_purchase_data_df['Price']-gender_norm.min())/(gender_norm.max()-gender_norm.min())
gender_norm_df = gender_norm_df.reset_index()
gender_purchase_data_df = gender_purchase_data_df.reset_index()
gender_purchase_data_df["normalized"] = gender_norm_df["Price"]

In [9]:
gender_purchase_data_df.head()



,Gender,Age,Item ID,Item Name,Price,SN,Age Demographics,normalized
0,Male,38,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39,0.584184
1,Male,21,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24,0.028061
2,Male,34,174,Primitive Blade,2.46,Assastnya25,30-34,0.158163
3,Male,21,92,Final Critic,1.36,Pheusrical25,20-24,0.380102
4,Male,23,63,Stormfury Mace,1.27,Aela59,20-24,0.364796


In [4]:
bins=[0,25,50,75,100]
bin_values=[0,9,14,19,24,29,34,39,44,49,100]
bin_labels=['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '>50']
purchase_data_df["Age Demographics"]= pd.cut(purchase_data_df["Age"], bin_values, labels=bin_labels)
purchase_data_df["Age Demographics"].head()

0    35-39
1    20-24
2    30-34
3    20-24
4    20-24
Name: Age Demographics, dtype: category
Categories (10, object): [<10 < 10-14 < 15-19 < 20-24 ... 35-39 < 40-44 < 45-49 < >50]

In [5]:
demographic_group = purchase_data_df.groupby("Age Demographics")
purchase_data_df.head()

,Age,Gender,Item ID,Item Name,Price,SN,Age Demographics
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34,35-39
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46,20-24
2,34,Male,174,Primitive Blade,2.46,Assastnya25,30-34
3,21,Male,92,Final Critic,1.36,Pheusrical25,20-24
4,23,Male,63,Stormfury Mace,1.27,Aela59,20-24


In [74]:
sn_price=purchase_data_df.groupby(["SN"])['Price'].sum()
sn_purchase=purchase_data_df.groupby(["SN"])['Price'].count()
sn_users=purchase_data_df.groupby(["SN"])
avg_sn= round(sn_price/sn_purchase,2)

top_sn=pd.DataFrame({"Purchase Count": sn_purchase, "Average Purchase Price":avg_sn, "Total Purchase Value":sn_price})
top_sn= top_sn.sort_values("Total Purchase Value", ascending=False)
top_sn= top_sn[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

top_sn.reset_index(inplace=True)
top_sn.round(2)
top_sn.head()

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
0,Undirrala66,5,3.41,17.06
1,Saedue76,4,3.39,13.56
2,Mindimnya67,4,3.18,12.74
3,Haellysu29,3,4.24,12.73
4,Eoda93,3,3.86,11.58


In [79]:
groupby_itemid = purchase_data_df.groupby('Item ID')
purchase_by_item = pd.DataFrame(groupby_itemid['Item ID'].count()) 
purchase_by_item.rename(columns = {"Item ID": "Purchase Count"}, inplace = True)

ttlpchsvlu = pd.DataFrame(groupby_itemid['Price'].sum()) 
ttlpchsvlu.rename(columns = {"Price": "Total Purchase Value"}, inplace =True)


no_dup_items = purchase_data_df.drop_duplicates('Item ID')

top5_pop = no_dup_items.merge(purchase_by_item, left_on = "Item ID", right_index = True)
top5_pop = top5_pop.merge(ttlpchsvlu, left_on = "Item ID", right_index = True)
top5_pop = top5_pop[['Item ID', "Item Name", "Price", "Purchase Count", "Total Purchase Value"]]
top5_pop.sort_values("Purchase Count", ascending = False, inplace = True)
top5_pop = top5_pop.iloc[0:5][:] 
top5_pop

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
61,39,"Betrayal, Whisper of Grieving Widows",2.35,11,25.85
116,84,Arcane Gem,2.23,11,24.53
81,175,Woeful Adamantite Claymore,1.24,9,11.16
35,13,Serenity,1.49,9,13.41
56,31,Trickster,2.07,9,18.63


In [81]:
top5_prof = no_dup_items.merge(purchase_by_item, left_on = "Item ID", right_index = True)
top5_prof = top5_prof.merge(ttlpchsvlu, left_on = "Item ID", right_index = True)
top5_prof = top5_prof[['Item ID', "Item Name", "Price", "Purchase Count", "Total Purchase Value"]]
top5_prof.sort_values("Total Purchase Value", ascending = False, inplace = True)
top5_prof.head()

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
57,34,Retribution Axe,4.14,9,37.26
107,115,Spectral Diamond Doomblade,4.25,7,29.75
50,32,Orenmir,4.95,6,29.70
100,103,Singed Scalpel,4.87,6,29.22
164,107,"Splitter, Foe Of Subtlety",3.61,8,28.88
